In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '../')

import fewshot.data
import fewshot.trainer

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models

import matplotlib.pyplot as plt
%matplotlib inline

# Refactoring tests on small dataset
We've refactored our data pipeline, utilites and training loop in the `fewshot` module. Let's test it below for a full run of training.

In [2]:
fewshot.data.fix_quotes('../data/small/styles.csv', '../data/small/styles_quoted.csv')

In [3]:
train_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [4]:
data = fewshot.data.FashionData('../data/small/styles_quoted.csv',
                                '../data/small/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=True)

In [5]:
n_classes = len(data.idx2name)
n_classes

20

In [6]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, n_classes)

In [8]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
fewshot.trainer.run_training(model, 5, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0
Training loss: 0.732
Validation loss: 1.123
Validation accuracy: 0.738
------------- epoch:  1


KeyboardInterrupt: 

In [ ]:
c1, c5, occ = fewshot.trainer.evaluate(model, data, 0, n_classes)

In [ ]:
c1

In [ ]:
c5

In [ ]:
occ

## Finetuning on the rare classes

In [ ]:
data = fewshot.data.FashionData('../data/small/styles_quoted.csv',
                                '../data/small/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=False)

In [ ]:
n_classes = len(data.idx2name)
n_classes

In [ ]:
num_ftrs = model.fc.in_features

## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, n_classes)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
fewshot.trainer.run_training(model, 5, data, optimizer, loss_func, gpu_id=0, root='./')

## Compare this to not finetuning:

In [ ]:
data = fewshot.data.FashionData('../data/small/styles_quoted.csv',
                                '../data/small/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=False)

In [ ]:
n_classes = len(data.idx2name)
n_classes

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, n_classes)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
fewshot.trainer.run_training(model, 5, data, optimizer, loss_func, gpu_id=0, root='./')